In [22]:
import streamlit as st
from PIL import Image
from keras.preprocessing.image import load_img, img_to_array
import numpy as np
from keras.models import load_model
import requests
from bs4 import BeautifulSoup
import webbrowser
import base64


In [23]:
model = load_model('FV.h5')

# Mapping of class indices to labels
labels = {
    0: 'apple', 1: 'banana', 2: 'beetroot', 3: 'capsicum', 4: 'cabbage', 
    5: 'capsicum', 6: 'carrot', 7: 'cauliflower', 8: 'chilli pepper', 9: 'corn', 
    10: 'cucumber', 11: 'eggplant', 12: 'garlic', 13: 'ginger', 14: 'grapes', 
    15: 'jalapeno', 16: 'kiwi', 17: 'lemon', 18: 'lettuce', 19: 'mango', 
    20: 'onion', 21: 'orange', 22: 'paprika', 23: 'pear', 24: 'peas', 
    25: 'pineapple', 26: 'pomegranate', 27: 'potato', 28: 'raddish', 
    29: 'soy beans', 30: 'spinach', 31: 'sweetcorn', 32: 'sweetpotato', 
    33: 'tomato', 34: 'turnip', 35: 'watermelon'
}
fruits = ['Apple','Banana','Chilli Pepper','Grapes','Jalapeno','Kiwi','Lemon','Mango','Orange','Paprika','Pear','Pineapple','Pomegranate','Watermelon']
vegetables = ['Beetroot','Cabbage','Capsicum','Carrot','Cauliflower','Corn','Cucumber','Eggplant','Ginger','Lettuce','Onion','Peas','Potato','Raddish','Soy Beans','Spinach','Sweetcorn','Sweetpotato','Tomato','Turnip']


In [24]:
# def fetch_calories(prediction):
#     try:
#         url = 'https://www.google.com/search?&q=calories in ' + prediction + ' 100g'
#         req = requests.get(url).text
#         scrap = BeautifulSoup(req, 'html.parser')
#         calories = scrap.find("div", class_="BNeawe iBp4i AP7Wnd").text
#         return calories
#     except Exception as e:
#         st.error("Unable to fetch the Calories.")
#         print(e)

In [25]:
def fetch_calories(prediction):
    try:
        # Construct the search URL
        url = 'https://www.google.com/search?&q=calories in ' + prediction + ' 100g'
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }

        # Make the request to Google
        req = requests.get(url, headers=headers)
        
        # Check if the response was successful
        if req.status_code != 200:
            print("Error: Unable to fetch data from Google.")
            return None
        
        # Parse the HTML content
        soup = BeautifulSoup(req.text, 'html.parser')
        
        # Find the calorie information (may need to adjust the selector based on the returned HTML)
        calories_info = soup.find_all("div", class_="BNeawe iBp4i AP7Wnd")

        # If calories_info is found, extract the relevant data
        if calories_info:
            for info in calories_info:
                text = info.get_text()
                if "calories" in text.lower():  # Check if it contains 'calories'
                    return text  # Return the text with calorie information
            print("Calories not found in the response.")
            return None
        else:
            print("No relevant data found.")
            return None
        
    except Exception as e:
        print("Error fetching calories:", e)
        return None

# Example usage
food_item = "apple"
calories = fetch_calories(food_item)
if calories:
    print(f"Calories in {food_item}: {calories}")

Calories in apple: 52 calories


In [26]:


def fetch_fat(prediction):
    try:
        url = 'https://www.google.com/search?&q=fat in ' + prediction
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        req = requests.get(url, headers=headers).text
        
        scrap = BeautifulSoup(req, 'html.parser')
        
        fat_info = scrap.find("div", class_="BNeawe iBp4i AP7Wnd")
        
        if fat_info:
            return fat_info.text
        else:
            print("Fat information not found.")
            return None
    except Exception as e:
        print("Error fetching fat information:", e)
        return None

# Example usage
food_item = "apple"
fat_content = fetch_fat(food_item)
if fat_content:
    print(f"Fat in {food_item}: {fat_content}")


Fat in apple: 0.2 g


In [27]:
def get_base64_of_bin_file(bin_file):
    """
    function to read png file 
    ----------
    bin_file: png -> the background image in local folder
    """
    with open(bin_file, 'rb') as f:
        data = f.read()
    return base64.b64encode(data).decode()



In [28]:
def set_bg_hack(main_bg):
    '''
    A function to unpack an image from root folder and set as bg.
 
    Returns
    -------
    The background.
    '''
    # set bg name
    main_bg_ext = "png"
        
    st.markdown(
         f"""
         <style>
         .stApp {{
             background: url(data:image/{main_bg_ext};base64,{base64.b64encode(open(main_bg, "rb").read()).decode()});
             background-size: cover
         }}
         </style>
         """,
         unsafe_allow_html=True
     )


In [29]:
def processed_img(img_path):
    img=load_img(img_path,target_size=(224,224,3))
    img=img_to_array(img)
    img=img/255
    img=np.expand_dims(img,[0])
    answer=model.predict(img)
    y_class = answer.argmax(axis=-1)
    print(y_class)
    y = " ".join(str(x) for x in y_class)
    y = int(y)
    res = labels[y]
    print(res)
    return res.capitalize()


In [30]:
def run():
    set_bg_hack(r"C:\Users\mas_r\OneDrive\Desktop\Major project\Images\bg_image.jpg")    
    st.title("Calorie Predictor & Diet Plan Recommender")


In [36]:
#     if st.button('Go to diet plans'):
#         webbrowser.open_new_tab(r"C:\Users\mas_r\OneDrive\Desktop\Major project\index1.html")
#     img_file = st.file_uploader("Choose an Image", type=["jpg", "png"])
#     if img_file is not None:
#         img = Image.open(img_file).resize((250,250))
#         st.image(img,use_column_width=False)
#         save_image_path = './upload_images/'+img_file.name
#         with open(save_image_path, "wb") as f:
#             f.write(img_file.getbuffer())

#         # if st.button("Predict"):
#         if img_file is not None:
#             result= processed_img(save_image_path)
#             print(result)
#             if result in vegetables:
#                 st.info('**Category : Vegetables**')
#             else:
#                 st.info('**Category : Fruit**')
#             st.success("**Predicted : "+result+'**')
#             cal = fetch_calories(result)
#             fat=fetch_fat(result)
#             if cal:
#                 st.warning('**'+cal+'(100 grams)**')
#             if fat:
#            	 st.warning('**'+'Total fat:'+fat+'**')
            
# run()
# def run():
#     # Streamlit app title
#     st.title("Calorie Predictor & Diet Plan Recommender")
    
#     # Button to navigate to diet plans
#     if st.button('Go to diet plans'):
#         webbrowser.open_new_tab(r"C:\Users\mas_r\OneDrive\Desktop\Major project\index1.html")
    
#     # File uploader for images
#     img_file = st.file_uploader("Choose an Image", type=["jpg", "png"])
    
#     if img_file is not None:
#         img = Image.open(img_file).resize((250, 250))
#         st.image(img, use_column_width=False)
        
#         # Save the uploaded image
#         save_image_path = './upload_images/' + img_file.name
#         with open(save_image_path, "wb") as f:
#             f.write(img_file.getbuffer())

#         # Process the image to get predictions
#         result = processed_img(save_image_path)
#         st.write("Predicted Result:", result)  # Debug output for predicted result

#         if result in vegetables:
#             st.info('**Category: Vegetables**')
#         else:
#             st.info('**Category: Fruits**')

#         st.success("**Predicted: " + result + '**')

#         # Fetch and display nutritional information
#         cal = fetch_calories(result)
#         fat = fetch_fat(result)

#         if cal:
#             st.warning('**' + cal + ' (100 grams)**')
#         if fat:
#             st.warning('**Total fat: ' + fat + '**')

# # Call the run function to execute the app
# if __name__ == "__main__":
#     run()

In [37]:
import streamlit as st
import os

def run():
    st.title("Image Upload")

    # Upload image
    uploaded_file = st.file_uploader("Choose an image...", type=["jpeg", "jpg", "png"])

    if uploaded_file is not None:
        # Define the directory to save uploaded images
        upload_dir = './upload_images'
        
        # Create the directory if it doesn't exist
        if not os.path.exists(upload_dir):
            os.makedirs(upload_dir)

        # Create the full path to save the file
        save_image_path = os.path.join(upload_dir, uploaded_file.name)

        # Log the current working directory and expected path
        st.write("Current Directory:", os.getcwd())
        st.write("Expected Path:", save_image_path)

        # Save the uploaded image
        try:
            with open(save_image_path, "wb") as f:
                f.write(uploaded_file.getbuffer())
            st.success("Image saved successfully!")
            st.image(save_image_path)
        except Exception as e:
            st.error(f"Error saving image: {e}")

if __name__ == "__main__":
    run()


1